In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

## close range vrs distance range relative to the trollinde radar.

In [2]:
track = pd.read_csv('../saved-files/threshold-10/all-cells-first_landfall-threshold-10.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,67,326.578302,280.675950,5,10,67,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,...,68.422543,14.734897,67,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082,True,0
1,0,68,336.121109,286.415064,196,10,68,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.396121e+06,...,68.331369,14.852404,68,0 days 00:00:00,111.375543,13.245867,34.567482,187.081946,True,0
2,0,78,359.923018,241.782637,166,10,78,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.419923e+06,...,68.145247,13.754067,78,0 days 00:00:00,152.158361,13.245867,34.567482,110.458189,False,5
3,1,87,360.498890,242.770250,205,10,188,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.420499e+06,...,68.139408,13.776453,78,0 days 00:05:00,152.149544,12.974534,35.577388,310.485962,True,0
4,1,14,236.134718,351.719741,3946,10,115,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.296135e+06,...,69.179040,16.675038,110,0 days 00:00:00,27.439561,12.974534,35.577388,3872.517749,False,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,297,32,269.998632,285.821708,142,10,24512,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.329999e+06,...,68.932863,14.974942,8469,0 days 00:05:00,53.353856,15.206985,38.598690,309.447699,True,0
1377,296,32,275.076364,319.000000,3,10,24472,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.335076e+06,...,68.858431,15.771137,8470,0 days 00:00:00,43.596674,15.390894,40.106802,36.815098,True,0
1378,297,34,278.872825,337.853842,25,10,24514,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.338873e+06,...,68.806038,16.218666,8477,0 days 00:00:00,49.197407,15.206985,38.598690,182.094241,True,0
1379,297,35,287.659006,397.126516,53,10,24515,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.347659e+06,...,68.661326,17.621146,8478,0 days 00:00:00,91.384398,15.206985,38.598690,70.651869,True,0


### Calculate distance relative to trollinde radar

In [3]:
distance = []
for i in range(len(track)):
    #lat_andenes = 69.141599 # latitude of Andenes
    lat_andenes = 69.2412 # latiude of trollinde radar
    #lon_andenes = 15.684104 # latitude of Andenes
    lon_andenes = 16.0030 # longitude of trollinde radar
    
    distances = tobac.analysis.haversine(track['latitude'][i], track['longitude'][i], lat_andenes, lon_andenes)
    distance.append(distances)

In [4]:
distance[:10]
# np.max(distance)

[104.41324626907375,
 111.37554259593695,
 152.15836079658845,
 152.14954354920306,
 27.439560675838116,
 28.715999713633266,
 29.374513300293657,
 31.038655822595178,
 33.25812603422149,
 35.86500933533962]

In [5]:
%%time
# apped distance to track dataframe
track['distance'] = pd.Series(distance)

CPU times: user 697 µs, sys: 0 ns, total: 697 µs
Wall time: 702 µs


In [6]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,67,326.578302,280.675950,5,10,67,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,...,68.422543,14.734897,67,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082,True,0
1,0,68,336.121109,286.415064,196,10,68,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.396121e+06,...,68.331369,14.852404,68,0 days 00:00:00,111.375543,13.245867,34.567482,187.081946,True,0
2,0,78,359.923018,241.782637,166,10,78,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.419923e+06,...,68.145247,13.754067,78,0 days 00:00:00,152.158361,13.245867,34.567482,110.458189,False,5
3,1,87,360.498890,242.770250,205,10,188,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.420499e+06,...,68.139408,13.776453,78,0 days 00:05:00,152.149544,12.974534,35.577388,310.485962,True,0
4,1,14,236.134718,351.719741,3946,10,115,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.296135e+06,...,69.179040,16.675038,110,0 days 00:00:00,27.439561,12.974534,35.577388,3872.517749,False,30


# close domain

In [7]:
# latitude of radar(reference point)
ref_latitude_radar = 69.2412
# longitude of radar
ref_longitude_radar = 16.0030
# first 50km from the trollinde radar on the top
latitude_top_50km = 69.6940 
# first 50km from the trollinde radar on the bottom
latitude_bottom_50km = 68.78878

latitude = ((track['latitude'] >= latitude_bottom_50km) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_top = ((track['latitude'] >= ref_latitude_radar) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_bottom = ((track['latitude'] <= ref_latitude_radar) & (track['latitude'] >= latitude_bottom_50km))

longitude = (track['longitude'] <= ref_longitude_radar)

mask_lat = (latitude  & longitude)

# mask_lat = (from_radar_to_50km_top | from_radar_to_50km_bottom)



close_domain_10 = track[ mask_lat]
close_domain_10

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
16,3,39,250.423362,318.045479,30,10,350,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.310423e+06,...,69.082940,15.808867,253,0 days 00:00:00,19.221204,12.708913,36.584485,37.809595,True,0
17,3,41,260.495928,309.864754,43,10,352,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.320496e+06,...,68.998970,15.583813,255,0 days 00:00:00,31.680540,12.708913,36.584485,39.799761,False,10
18,4,46,260.955610,310.205639,54,10,457,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.320956e+06,...,68.994497,15.591049,255,0 days 00:05:00,31.958503,12.845599,38.092506,67.658450,False,5
19,5,43,259.637361,310.448155,8,10,562,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.319637e+06,...,69.006240,15.600140,255,0 days 00:10:00,30.650596,13.138674,36.584485,18.903445,True,0
27,6,55,280.125754,284.112193,27,10,680,2020-03-13 04:30:00,2020-03-13 04:30:00,-2.340126e+06,...,68.842200,14.911974,393,0 days 00:00:00,62.126223,13.454442,38.092506,30.845914,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,292,37,274.435767,286.732912,4,10,24300,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.334436e+06,...,68.891832,14.987730,8419,0 days 00:00:00,56.060765,14.985573,38.596077,26.864266,True,0
1374,295,34,249.731863,316.096219,22,10,24428,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.309732e+06,...,69.091005,15.762732,8459,0 days 00:00:00,19.235877,15.270663,40.109338,118.403926,True,0
1375,296,31,270.049321,283.144073,146,10,24471,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.330049e+06,...,68.934506,14.909480,8469,0 days 00:00:00,55.255951,15.390894,40.106802,NaN,False,5
1376,297,32,269.998632,285.821708,142,10,24512,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.329999e+06,...,68.932863,14.974942,8469,0 days 00:05:00,53.353856,15.206985,38.598690,309.447699,True,0


In [8]:
%%time
cells_relative_time_to_radar_close = [];

for index, cell_id in enumerate(np.unique(close_domain_10['cell'])):
    if cell_id:
        # get all the closed domain cell id
        cells = close_domain_10[close_domain_10['cell'] == cell_id]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar = all_times[::-1][0]
        relative_time = np.subtract(time_cell_hit_radar, all_times)
        # convert to seconds 
        relative_time = relative_time.seconds // 60
        # print(cell_id, relative_time)
        cells_relative_time_to_radar_close.append(relative_time)

CPU times: user 87.3 ms, sys: 609 µs, total: 87.9 ms
Wall time: 88.2 ms


In [9]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_close = pd.Series([item for sublist in cells_relative_time_to_radar_close for item in sublist])
close_domain_10['relative-time-to-LF-close'] = cells_relative_times_to_radar_close.values

/glade/scratch/noteng/ipykernel_42924/4114636113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_domain_10['relative-time-to-LF-close'] = cells_relative_times_to_radar_close.values


In [10]:
close_domain_10 = close_domain_10.reset_index(drop=True)

In [11]:
close_domain_10.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time,relative-time-to-LF-close
0,3,39,250.423362,318.045479,30,10,350,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.310423e+06,...,15.808867,253,0 days 00:00:00,19.221204,12.708913,36.584485,37.809595,True,0,0
1,3,41,260.495928,309.864754,43,10,352,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.320496e+06,...,15.583813,255,0 days 00:00:00,31.680540,12.708913,36.584485,39.799761,False,10,10
2,4,46,260.955610,310.205639,54,10,457,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.320956e+06,...,15.591049,255,0 days 00:05:00,31.958503,12.845599,38.092506,67.658450,False,5,5
3,5,43,259.637361,310.448155,8,10,562,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.319637e+06,...,15.600140,255,0 days 00:10:00,30.650596,13.138674,36.584485,18.903445,True,0,0
4,6,55,280.125754,284.112193,27,10,680,2020-03-13 04:30:00,2020-03-13 04:30:00,-2.340126e+06,...,14.911974,393,0 days 00:00:00,62.126223,13.454442,38.092506,30.845914,True,0,0


In [12]:
len(close_domain_10)

259

# distant domain

In [13]:
ref_latitude_point_top = 69.6940 
ref_latitude_point_bottom = 68.78878
# longitude of radar
ref_longitude_radar1 = 16.0030
latitude_top_50km_to_150km = 70.5978 # 100km from first 50km on the top
latitude_bottom_50km_to_150km = 67.8878 # 100km from first 50km on the bottom

latitude1 = ((track['latitude'] >= latitude_bottom_50km_to_150km) & (track['latitude'] <= latitude_top_50km_to_150km))
longitude1 = (track['longitude'] <= ref_longitude_radar1)

mask_lat = (latitude1 & longitude1)


distant_domain_10 = track[ mask_lat ]
distant_domain_10

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,67,326.578302,280.675950,5,10,67,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,...,68.422543,14.734897,67,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082,True,0
1,0,68,336.121109,286.415064,196,10,68,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.396121e+06,...,68.331369,14.852404,68,0 days 00:00:00,111.375543,13.245867,34.567482,187.081946,True,0
2,0,78,359.923018,241.782637,166,10,78,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.419923e+06,...,68.145247,13.754067,78,0 days 00:00:00,152.158361,13.245867,34.567482,110.458189,False,5
3,1,87,360.498890,242.770250,205,10,188,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.420499e+06,...,68.139408,13.776453,78,0 days 00:05:00,152.149544,12.974534,35.577388,310.485962,True,0
11,1,55,291.000000,279.391287,3,10,156,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.351000e+06,...,68.747008,14.775128,144,0 days 00:00:00,73.664960,12.974534,35.577388,NaN,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,292,37,274.435767,286.732912,4,10,24300,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.334436e+06,...,68.891832,14.987730,8419,0 days 00:00:00,56.060765,14.985573,38.596077,26.864266,True,0
1374,295,34,249.731863,316.096219,22,10,24428,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.309732e+06,...,69.091005,15.762732,8459,0 days 00:00:00,19.235877,15.270663,40.109338,118.403926,True,0
1375,296,31,270.049321,283.144073,146,10,24471,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.330049e+06,...,68.934506,14.909480,8469,0 days 00:00:00,55.255951,15.390894,40.106802,NaN,False,5
1376,297,32,269.998632,285.821708,142,10,24512,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.329999e+06,...,68.932863,14.974942,8469,0 days 00:05:00,53.353856,15.206985,38.598690,309.447699,True,0


In [14]:
%%time
cells_relative_time_to_radar_distant = [];

for index, cell_idx in enumerate(np.unique(distant_domain_10['cell'])):
    if cell_idx:
        # get all the closed domain cell id
        cells = distant_domain_10[distant_domain_10['cell'] == cell_idx]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar1 = all_times[::-1][0]
        relative_time1 = np.subtract(time_cell_hit_radar1, all_times)
        # convert to seconds 
        relative_time1 = relative_time1.seconds // 60
        # print(cell_idx, relative_time1)
        cells_relative_time_to_radar_distant.append(relative_time1)

CPU times: user 315 ms, sys: 0 ns, total: 315 ms
Wall time: 335 ms


In [15]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_distant = pd.Series([item for sublist in cells_relative_time_to_radar_distant for item in sublist])
distant_domain_10['relative-time-to-LF-distant'] = cells_relative_times_to_radar_distant.values

/glade/scratch/noteng/ipykernel_42924/2184313546.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distant_domain_10['relative-time-to-LF-distant'] = cells_relative_times_to_radar_distant.values


In [16]:
distant_domain_10 = distant_domain_10.reset_index(drop=True)


In [17]:
distant_domain_10.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time,relative-time-to-LF-distant
0,0,67,326.578302,280.675950,5,10,67,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,...,14.734897,67,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082,True,0,0
1,0,68,336.121109,286.415064,196,10,68,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.396121e+06,...,14.852404,68,0 days 00:00:00,111.375543,13.245867,34.567482,187.081946,True,0,0
2,0,78,359.923018,241.782637,166,10,78,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.419923e+06,...,13.754067,78,0 days 00:00:00,152.158361,13.245867,34.567482,110.458189,False,5,5
3,1,87,360.498890,242.770250,205,10,188,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.420499e+06,...,13.776453,78,0 days 00:05:00,152.149544,12.974534,35.577388,310.485962,True,0,0
4,1,55,291.000000,279.391287,3,10,156,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.351000e+06,...,14.775128,144,0 days 00:00:00,73.664960,12.974534,35.577388,NaN,True,0,0


In [18]:
len(distant_domain_10)

899

In [19]:
len(close_domain_10)

259

In [20]:
len(track)

1381

In [21]:
len(close_domain_10) + len(distant_domain_10)

1158

In [22]:
distant_domain_10['latitude'].max(), distant_domain_10['latitude'].min(),close_domain_10['latitude'].max(), close_domain_10['latitude'].min()

(70.01310351388585, 67.90266442704792, 69.69024056002493, 68.78884335830335)

In [23]:
close_domain_10.to_csv('../saved-files/threshold-10/close-domain-threshold-10-LF.csv', index=False)
distant_domain_10.to_csv('../saved-files/threshold-10/distant-domain-threshold-10-LF.csv', index=False)

# END

In [42]:
# max_dBZ_10 = []
# for indexes, time in enumerate(np.unique(close_domain_10['relative-time-to-radar'])):
#     # Sorting based on same relative times
#     sort_relative_times_threshold_10 = close_domain_10[close_domain_10['relative-time-to-radar'] == time]
#     # Find each relative time associated with a particular reflectivity value
    
#     max_dbZ_threshold_10 = np.mean(sort_relative_times_threshold_10['max dBZ'])
#     max_dBZ_10.append(max_dbZ_threshold_10)